In [86]:
import sys
sys.path.append("..")
from logging import DEBUG
from logprep.factory import Factory
from logprep.registry import Registry
from logprep.abc import Processor
from logprep.processor.dropper.rule import DropperRule
from time import perf_counter
from logging import getLogger

class OldDropper(Processor):
    """Normalize log events by copying specific values to standardized fields."""

    rule_class = DropperRule

    def _traverse_dict_and_delete(self, dict_: dict, sub_fields, drop_full: bool):
        sub_field = sub_fields[0] if sub_fields else None
        remaining_sub_fields = sub_fields[1:]

        if not remaining_sub_fields and sub_field in dict_.keys():
            del dict_[sub_field]
        if (
            isinstance(dict_, dict)
            and sub_field in dict_
            and (isinstance(dict_[sub_field], dict) and dict_[sub_field])
        ):
            self._traverse_dict_and_delete(dict_[sub_field], remaining_sub_fields, drop_full)
            if dict_[sub_field] == {} and drop_full:
                del dict_[sub_field]

    def _drop_field(self, event: dict, dotted_field: str, drop_full: bool):
        sub_fields = dotted_field.split(".")
        self._traverse_dict_and_delete(event, sub_fields, drop_full)

    def _apply_rules(self, event: dict, rule: DropperRule):
        """Drops fields from event Logs."""

        if self._logger.isEnabledFor(DEBUG):  # pragma: no cover
            self._logger.debug(f"{self.describe()} processing matching event")
        for drop_field in rule.fields_to_drop:
            self._try_dropping_field(event, drop_field, rule.drop_full)

    def _try_dropping_field(self, event: dict, dotted_field: str, drop_full: bool):
        if self._field_exists(event, dotted_field):
            self._drop_field(event, dotted_field, drop_full)

dropper_config = {
    "dropper": {
        "type": "dropper",
        "specific_rules": ["/dev"],
        "generic_rules": ["/dev"]  
    }
}

logger = getLogger(__name__)

dropper = Factory.create(dropper_config, logger)

Registry.mapping.update({"old_dropper": OldDropper})

old_dropper_config = {
    "dropper": {
        "type": "old_dropper",
        "specific_rules": ["/dev"],
        "generic_rules": ["/dev"]  
    }
}

old_dropper = Factory.create(old_dropper_config, logger)

dropper
old_dropper

rules = [
{
    "filter": "key1",
    "dropper": {
        "drop": ["key1"],
        
    },
},
{
    "filter": "key1",
    "dropper": {
        "drop": ["key1.a", "key1.b", "key1.key2.a", "key1.key2.key3.b"],
        "drop_full": False
    }
},
{
    "filter": "key1",
    "dropper": {
        "drop": ["key1.key2.a", "key1.key2.key3.key4.key5.a", "key1.key2.ke3.b", "key1.key2.ke3.a"],
    }
}

]

rules = [DropperRule._create_from_dict(rule) for rule in rules]
[dropper._specific_tree.add_rule(rule) for rule in rules]
[old_dropper._specific_tree.add_rule(rule) for rule in rules]

dropper._rules


In [87]:
document = {
    "list": ["existing"],
    "key1": {
        "a": {"b": "value"},
        "key2": {
            "a": {"b": "value"},
            "key3": {
                "a": {"b": "value"},
                "key4": {
                    "a": {"b": "value"},
                    "key5": {
                        "a": {"b": "value"},
                        "key6": "value"}
                    
}}}}}


In [88]:
documents = [document for _ in range(5_000_000)]
start = perf_counter()
for event in documents:
    dropper.process(event)
print(perf_counter() - start)


KeyboardInterrupt: 

In [ ]:
documents1 = [document for _ in range(5_000_000)]
start = perf_counter()
for event in documents1:
    old_dropper.process(event)
print(perf_counter() - start)

2.1645570440000483
